## EDA

The notebook demonstrates the raw annotations and the derived keypoints, utilized in the model training process.

In [ ]:
import os
import cv2
import json
import numpy as np
from collections import defaultdict

import matplotlib.pyplot as plt

from src.datatools.reader import read_annot
from src.datatools.intersections import get_intersections

%load_ext autoreload
%autoreload 2

In [ ]:
sample_id = '00016'
img_path = f'/workdir/data/dataset/valid/{sample_id}.jpg'
annot_path = f'/workdir/data/dataset/valid/{sample_id}.json'

In [ ]:
# Print the sample annotation
read_annot(annot_path)

In [ ]:
IMG_SIZE = (960, 540)

POINT_COLOR = (0, 0, 255)
INTERSECTION_COLOR = (255, 0, 0)
CIRCLE_COLOR = (0, 255, 255)

LINE_COLORS = {
    "Big rect. left bottom": (45, 175, 80),
    "Big rect. left main": (40, 175, 170),
    "Big rect. left top": (255, 175, 65),
    "Big rect. right bottom": (80, 0, 50),
    "Big rect. right main": (70, 0, 40),
    "Big rect. right top": (255, 0, 50),
    "Circle central": (255, 0, 0),
    "Circle left": (255, 127, 0),
    "Circle right": (255, 0, 127),
    "Goal left crossbar": (0, 0, 255),
    "Goal left post left ": (0, 127, 255),
    "Goal left post right": (127, 0, 255),
    "Goal right crossbar": (0, 0, 255),
    "Goal right post left": (0, 255, 255),
    "Goal right post right": (255, 0, 255),
    "Middle line": (0, 0, 0),
    "Side line bottom": (65, 85, 150),
    "Side line left": (50, 255, 255),
    "Side line right": (85, 60, 0),
    "Side line top": (255, 85, 65),
    "Small rect. left bottom": (0, 127, 127),
    "Small rect. left main": (127, 127, 127),
    "Small rect. left top": (127, 127, 0),
    "Small rect. right bottom": (0, 127, 127),
    "Small rect. right main": (127, 127, 127),
    "Small rect. right top": (127, 127, 0)
}

In [ ]:
# Blue circles - actual points, used in model training and prediucted
# Small red dots - points from annotation as-is
def draw_annot(img, annot, draw_circles=False):
    h, w, _ = img.shape
    for cls in annot:
        points = annot[cls]
        points = [(int(round(point[0]*w)), int(round(point[1]*h))) for point in points]
        for point in points:
            cv2.circle(img, point, 1, POINT_COLOR, 1)
            if draw_circles:
                cv2.circle(img, point, 5, CIRCLE_COLOR, 1)
        if len(points) > 1:
            for i in range(len(points)-1):
                cv2.line(img, points[i], points[i+1], LINE_COLORS[cls], 1, cv2.LINE_AA)
    inters, _ = get_intersections(annot)
    print(inters)
    for point in inters.values():
        if point is not None:
            point = (int(round(point[0])), int(round(point[1])))
            cv2.circle(img, point, 4, INTERSECTION_COLOR, 1)
    return img

In [ ]:
def show_img(img):
    plt.figure(dpi=150)
    plt.imshow(img[:, :, ::-1])  # BGR2RGB
    plt.show()

In [ ]:
img = cv2.imread(img_path)
annot = read_annot(annot_path)
annot_img = draw_annot(img, annot, False)
show_img(annot_img)